プログラム作成に当たって、以下のソースコードを大いに参考にしました。

https://github.com/msiplab/VieWork.git

copyright : 2020 Shogo Muramatsu

## 配列の準備
数値計算ライブラリ NumPy を利用

In [72]:
import numpy as np
import pandas as pd

#numpyのrandom関数で2桁の整数を要素とする16Ｘ16の行列を生成した。これを入力配列とする。
#入力配列
arrayX = np.matrix(
 [[2,6,2,4],
  [2,2,2,6],
  [6,6,4,4],
  [2,2,2,4]])

#配列Z
arrayZ = np.matrix(
 [[1,1,1,0],
  [1,1,0,0],
  [1,0,0,0],
  [0,0,0,0]])

#display(arrayX)

## 離散コサイン変換行列の定義

\begin{array}{l}
y[m]=\sqrt{\frac{2}{M}} k_{m} \sum_{n=0}^{N-1} \cos \left(\frac{(2 n+1) m \pi}{2 M}\right) x[n],\, m=0,1, \cdots, M-1 \\
x[n]=\sqrt{\frac{2}{M}} \sum_{m=0}^{N-1} k_{m} \cos \left(\frac{(2 n+1) m \pi}{2 M}\right) y[m],\, n=0,1, \cdots, M-1
\end{array}
ただし、
\begin{equation}
k_{m}=\left\{\begin{array}{cc}
\frac{1}{\sqrt{2}} & m=0 \\
1 & m=1,2, \cdots, M-1
\end{array}\right.
\end{equation}

### 順変換行列


In [73]:
dctSize = 4
arrayDct = np.matrix(np.zeros([dctSize,dctSize]))
for iCol in range(dctSize):
    for iRow in range(dctSize):
        k = 1
        if iRow == 0:
            k = 1/np.sqrt(2)
        arrayDct[iRow,iCol] = np.sqrt(2/dctSize)*k*np.cos((2*iCol+1)*iRow*np.pi/(2*dctSize))

#display(arrayDct)

### 逆変換行列

離散コサイン変換(DCT)は正規直交行列のため、逆変換行列は順変換行列の転置で与えられる。
\begin{equation}
\mathbf{C}^{-1}=\mathbf{C}^T
\end{equation}

In [74]:
#逆変換行列
arrayIdct = arrayDct.T

#display(arrayIdct)

## 二次元DCT
\begin{equation}
\mathbf{Y}=\mathbf{CXC}^T
\end{equation}


In [75]:
#a. 右の配列𝐗 ∈ ℝ4×4に対し、二次元DCTを施し、変換係数𝐘 を求めよ。
arrayY = np.dot(np.dot(arrayDct,arrayX),arrayDct.T)

df = pd.DataFrame(arrayY)
display(df)

,0,1,2,3
0,14.000000,-1.148050,1.000000,-2.771639
1,0.224171,-0.500000,-0.765367,-1.792893
2,-2.000000,-0.382683,-1.000000,-0.923880
3,3.154322,3.207107,-1.847759,-0.500000


In [76]:
#b. a.の結果に対し、右の配列Zを係数毎にかけ算せよ。
#上記で求めた変換係数Y(二次元DCTの実行結果arrayY)に基底画像B(arrayZ)を乗算する 二次元配列に二次元配列をいれている
arrayYh = []
for i in range(dctSize):
  for j in range(dctSize):
    a = arrayY[i,j] * arrayZ
    arrayYh.append(a)

display(arrayYh)

[matrix([[14., 14., 14.,  0.],
         [14., 14.,  0.,  0.],
         [14.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.]]),
 matrix([[-1.1480503, -1.1480503, -1.1480503, -0.       ],
         [-1.1480503, -1.1480503, -0.       , -0.       ],
         [-1.1480503, -0.       , -0.       , -0.       ],
         [-0.       , -0.       , -0.       , -0.       ]]),
 matrix([[1., 1., 1., 0.],
         [1., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.]]),
 matrix([[-2.7716386, -2.7716386, -2.7716386, -0.       ],
         [-2.7716386, -2.7716386, -0.       , -0.       ],
         [-2.7716386, -0.       , -0.       , -0.       ],
         [-0.       , -0.       , -0.       , -0.       ]]),
 matrix([[0.22417076, 0.22417076, 0.22417076, 0.        ],
         [0.22417076, 0.22417076, 0.        , 0.        ],
         [0.22417076, 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ]]),
 matrix([[-0.5, -0.5, -0.5, -0. ],
         [-0.5,

## 二次元逆DCT
\begin{equation}
\mathbf{X}=\mathbf{C}^{-1}\mathbf{YC}^{-T}
\end{equation}

In [77]:
#c. b.の結果に対し、二次元逆DCTを施し、元の配列𝐗の(ここではarrayR)を求めよ。
arrayR = []
for i in range(dctSize):
  for j in range(dctSize):
    a = np.dot(np.dot(arrayIdct,arrayY[i,j]),arrayIdct.T)
    arrayR.append(a)
display(arrayR)

#d. c.の近似arraを小数点以下第１位で四捨五入し、整数化した配列を求めよ。
apro_arrayR = np.round(arrayR, decimals=0) #小数点第一位を四捨五入して整数化
display(apro_arrayR)

[matrix([[ 1.40000000e+01, -1.44264514e-15, -1.61898863e-15,
          -6.78824974e-16],
         [-1.40744826e-15,  1.40000000e+01,  4.43661358e-16,
           1.47106075e-15],
         [-1.29367649e-15, -4.26684414e-16,  1.40000000e+01,
          -5.46292844e-16],
         [-9.30911998e-16,  2.02287566e-15, -1.18168794e-15,
           1.40000000e+01]]),
 matrix([[-1.14805030e+00,  1.20501732e-16,  1.15012137e-16,
           1.02956429e-16],
         [ 5.77979942e-17, -1.14805030e+00, -1.07379125e-17,
          -1.27186933e-16],
         [ 1.13309145e-16,  1.07379125e-17, -1.14805030e+00,
           3.93465205e-17],
         [ 7.29769589e-17, -1.29010564e-16,  3.22745729e-17,
          -1.14805030e+00]]),
 matrix([[ 1.00000000e+00, -7.20205172e-17, -1.26842198e-16,
          -1.02877618e-16],
         [-9.06864831e-17,  1.00000000e+00, -5.14328292e-17,
           1.26950890e-16],
         [-8.04206565e-17, -5.14328292e-17,  1.00000000e+00,
          -6.68660531e-17],
         [-1.0287

array([[[14., -0., -0., -0.],
        [-0., 14.,  0.,  0.],
        [-0., -0., 14., -0.],
        [-0.,  0., -0., 14.]],

       [[-1.,  0.,  0.,  0.],
        [ 0., -1., -0., -0.],
        [ 0.,  0., -1.,  0.],
        [ 0., -0.,  0., -1.]],

       [[ 1., -0., -0., -0.],
        [-0.,  1., -0.,  0.],
        [-0., -0.,  1., -0.],
        [-0.,  0., -0.,  1.]],

       [[-3.,  0.,  0.,  0.],
        [ 0., -3., -0., -0.],
        [ 0., -0., -3.,  0.],
        [ 0., -0.,  0., -3.]],

       [[ 0., -0., -0., -0.],
        [-0.,  0.,  0.,  0.],
        [-0.,  0.,  0., -0.],
        [-0.,  0., -0.,  0.]],

       [[-1.,  0.,  0.,  0.],
        [ 0., -1., -0., -0.],
        [ 0.,  0., -1.,  0.],
        [ 0., -0.,  0., -1.]],

       [[-1.,  0.,  0.,  0.],
        [ 0., -1.,  0., -0.],
        [ 0.,  0., -1.,  0.],
        [ 0., -0.,  0., -1.]],

       [[-2.,  0.,  0.,  0.],
        [ 0., -2.,  0., -0.],
        [ 0., -0., -2.,  0.],
        [ 0., -0.,  0., -2.]],

       [[-2.,  0.,  0., 

In [78]:
#元の配列𝐗とd.の結果との差を求めよ
arrayDif = []
for i in range(dctSize):
  for j in range(dctSize):
    dif = abs(arrayX[i,j]) - abs(apro_arrayR[i,j])
    arrayDif.append(dif)
display(arrayDif)

[array([-12.,   2.,   2.,   2.]),
 array([ 6., -8.,  6.,  6.]),
 array([  2.,   2., -12.,   2.]),
 array([  4.,   4.,   4., -10.]),
 array([1., 2., 2., 2.]),
 array([2., 1., 2., 2.]),
 array([2., 2., 1., 2.]),
 array([6., 6., 6., 5.]),
 array([5., 6., 6., 6.]),
 array([6., 5., 6., 6.]),
 array([4., 4., 3., 4.]),
 array([4., 4., 4., 3.]),
 array([-1.,  2.,  2.,  2.]),
 array([ 2., -1.,  2.,  2.]),
 array([ 2.,  2., -1.,  2.]),
 array([4., 4., 4., 1.])]